In [27]:
import tensorflow 
#import tensorflow.keras.models
#from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [28]:
##loading the models and files
model = tensorflow.keras.models.load_model('model.h5')

In [7]:
#loading scalers and incoded
with open('lable_encoded_gender.pkl','rb') as file:
    lable_encoded_gender = pickle.load(file)
with open('onehot_encoded_Geoghraphy.pkl','rb') as file:
    onehot_encoded_Geography = pickle.load(file)
with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)

In [17]:
#example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure':3,
    'Balance':10000,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':20000
}

In [18]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,10000,1,1,1,20000


In [19]:
#data preprocessing
input_df['Gender'] = lable_encoded_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,10000,1,1,1,20000


In [20]:
#encode onehot encoded 
geo_encoded = onehot_encoded_Geography.transform([input_df['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoded_Geography.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Administrator\Desktop\NLP Learning\annclassification\ann_env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,10000,1,1,1,20000,1.0,0.0,0.0


In [22]:
input_scaled = scalar.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.05836066,
        -0.91668767,  0.64920267,  0.97481699, -1.39843232,  1.00150113,
        -0.57946723, -0.57638802]])

In [23]:
#Predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.22084065]], dtype=float32)

In [24]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.22084065)

In [25]:
if prediction_proba > 0.5:
    print("the customer is likely to churn")
else:
    print("the customer is not likely to churn")

the customer is not likely to churn
